## Ocorrências aeronáuticas na aviação civil brasileira
## Data modeling

Este notebook parte do notebook de exploração de dados para realizar a **classificação de ocorrências aéreas entre acidentes e incidentes**. Uma vez importados os dados, operações de feature engineering são realizadas, sobretudo aquelas para conciliação entre os datasets de ocorrências e de aeronaves, os quais dispõem de entidades básicas diferentes. Então, o procedimento de modelagem inicia com o pré-processamento dos dados, para que então diferentes modelos de classificação sejam treinados a partir de diferentes algoritmos de aprendizado.

Quanto às operações de **pré-processamento**, missings são avaliados e features são classificadas entre numéricas, binárias e categóricas, a partir do que as numéricas podem receber a transformação logarítmica e a padronização, enquanto que as categóricas implicam na transformação de one-hot encoding. Além disso, o ocorre o tratamento de missings com a imputação de 0 e a criação de variáveis binárias indicando a ocorrência de um missing para uma dada variável.

O treinamento de modelos parte dos seguintes algoritmos de aprendizado:
* [Regressão logística com regularização L1](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html):
    * Hiper-parâmetro otimizado via grid search: parâmetro de regularização.


* [SVM com kernel polinomial](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html):
    * Hiper-parâmetro otimizado via grid search: grau máximo do polinômio.
    

* [LightGBM](https://lightgbm.readthedocs.io/en/latest/) e [XGBoost](https://xgboost.readthedocs.io/en/stable/):
    * Hiper-parâmetros otimizados via random search: subsample, maximum depth, learning rate e número de estimadores do ensemble.
    
Uma vez que os hiper-parâmetros sejam otimizados via K-folds cross-validation nos dados de treinamento, os modelos finais são avaliados no dataset de teste, o que permite a escolha do melhor modelo, bem como a otimização do limiar. Quanto às métricas de performance avaliadas, o [ROC-AUC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html) foi calculado para otimização dos hiper-parâmetros, enquanto que o limiar segue da otimização do [Matthews correlation coefficient (MCC)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html).

**Sumário:**
1. [Bibliotecas](#libraries)<a href='#libraries'></a>.
2. [Funções e classes](#functions_classes)<a href='#functions_classes'></a>.
3. [Configurações](#settings)<a href='#settings'></a>.
4. [Importando os dados](#imports)<a href='#imports'></a>.
    * [Feature engineering](#feat_eng)<a href='#feat_eng'></a>.


5. [Modelagem](#data_modeling)<a href='#data_modeling'></a>.
    * [Pré-processamento dos dados](#pre_proc)<a href='#pre_proc'></a>.
    * [Regressão logística](#lr)<a href='#lr'></a>.
    * [SVM](#svm)<a href='#svm'></a>.
    * [LightGBM](#lightgbm)<a href='#lightgbm'></a>.
    * [XGBoost](#xgboost)<a href='#xgboost'></a>.
    * [Model selection](#model_selection)<a href='#model_selection'></a>.

<a id='libraries'></a>

## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import matthews_corrcoef, confusion_matrix, accuracy_score
from scipy.stats import norm, uniform, randint

<a id='functions_classes'></a>

## Funções e classes

In [2]:
from utils import train_test_split, running_time
from transformations import applying_one_hot
from pre_process import pre_process
from kfolds import Kfolds_fit
from features_selection import FeaturesSelection

<a id='settings'></a>

## Configurações

In [3]:
# Indique se resultados devem ser exportados:
export = True

# Indique se as variáveis numéricas devem ser transformadas:
numerical_transf = True

# Descreve o teste a ser executado:
comment = ''

<a id='imports'></a>

## Importando os dados

<a id='ocorr'></a>

### Dados de ocorrência

In [4]:
ocorr = pd.read_csv('../Datasets/ocorrencia.csv', sep=';')

# Criando o label de acidente:
ocorr['y'] = ocorr.ocorrencia_classificacao.apply(lambda x: 1 if 'acidente' in x.lower() else 0)

# Modificando as variáveis de data e hora:
ocorr['ocorrencia_dia'] = ocorr['ocorrencia_dia'].apply(lambda x: x.split('/')[2]+'-'+x.split('/')[1]+'-'+\
                                                            x.split('/')[0])
ocorr['ocorrencia_dia'] = ocorr['ocorrencia_dia'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
ocorr['weekday'] = ocorr['ocorrencia_dia'].apply(lambda x: str(x.weekday()))
ocorr['faixa_hora'] = ocorr.ocorrencia_hora.apply(lambda x: str(x).split(':')[0] if len(str(x).split(':')) > 0
                                                  else np.NaN)

print(f'Shape de ocorr: {ocorr.shape}.')
print(f'Número de observações distintas: {ocorr.codigo_ocorrencia.nunique()}.')
print(f'Intervalo de tempo: ({ocorr.ocorrencia_dia.min()}, {ocorr.ocorrencia_dia.max()}).')

# Conjunto de variáveis de suporte:
drop_vars = ['codigo_ocorrencia', 'codigo_ocorrencia1', 'codigo_ocorrencia2', 'codigo_ocorrencia3',
             'codigo_ocorrencia4', 'ocorrencia_classificacao', 'ocorrencia_aerodromo',
             'investigacao_aeronave_liberada', 'investigacao_status', 'divulgacao_relatorio_numero',
             'divulgacao_relatorio_publicado', 'divulgacao_dia_publicacao', 'total_recomendacoes', 'y',
             'ocorrencia_latitude', 'ocorrencia_longitude', 'ocorrencia_dia', 'ocorrencia_hora']

# Forçando o sorting temporal:
ocorr.sort_values(['ocorrencia_dia', 'ocorrencia_hora'], ascending=[True, True], inplace=True)
ocorr.reset_index(drop=True, inplace=True)

ocorr.head(3)

Shape de ocorr: (6114, 25).
Número de observações distintas: 6114.
Intervalo de tempo: (2010-01-03 00:00:00, 2021-08-18 00:00:00).


,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,y,weekday,faixa_hora
0,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,0,6,03
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,...,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,0,6,11
2,40211,40211,40211,40211,40211,INCIDENTE,***,***,RIO DE JANEIRO,RJ,...,FINALIZADA,***,NÃO,NaN,0,1,NÃO,0,6,12


#### Tratando missings identificados por "***"

In [5]:
# Loop sobre colunas:
for c in list(ocorr.columns):
    # Checando a existência de missings identificados por "***":
    if '***' in list(ocorr[c].unique()):
        # Modificando a definição do missing:
        ocorr[c] = ocorr[c].apply(lambda x: 'missing_value' if x=='***' else x)

#### Train-test split

In [6]:
ocorr_train, ocorr_test = train_test_split(ocorr, preserve_date=True, date_var='ocorrencia_dia', test_ratio=0.3,
                                           shuffle=False, seed=1)

<a id='aero'></a>

### Dados de aeronave

In [7]:
aero = pd.read_csv('../Datasets/aeronave.csv', sep=';')

# Corrigindo a variável de ano de fabricação:
aero['aeronave_ano_fabricacao'] = aero.aeronave_ano_fabricacao.apply(lambda x: x if x in range(1900, 2100, 1)
                                                                     else np.NaN)

print(f'Shape de aero: {aero.shape}.')
print(f'Número de observações distintas: {aero.codigo_ocorrencia2.nunique()}.')

# Conjunto de variáveis de suporte:
drop_vars_aero = ['aeronave_modelo', 'aeronave_matricula', 'aeronave_tipo_icao', 'aeronave_pmd',
                  'aeronave_pmd_categoria', 'aeronave_nivel_dano', 'aeronave_fatalidades_total',
                  'aeronave_pais_fabricante', 'aeronave_registro_categoria']

aero.head(3)

Shape de aero: (6188, 23).
Número de observações distintas: 6114.


,codigo_ocorrencia2,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,...,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total
0,39115,PTNQX,***,AVIÃO,NEIVA INDUSTRIA AERONAUTICA,EMB-711A,P28R,PISTÃO,MONOMOTOR,1202,...,BRASIL,BRASIL,AVIÃO,PARTICULAR,BRIGADEIRO ARARIPE MACEDO,CORRENTINA,DECOLAGEM,PRIVADA,SUBSTANCIAL,0
1,39155,PTLVI,***,AVIÃO,BEECH AIRCRAFT,C90,BE9L,TURBOÉLICE,BIMOTOR,4377,...,BRASIL,BRASIL,AVIÃO,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,DECOLAGEM,PRIVADA,NENHUM,0
2,39156,PPPTO,***,AVIÃO,AEROSPATIALE AND ALENIA,ATR-72-212A,AT72,TURBOÉLICE,BIMOTOR,22500,...,BRASIL,BRASIL,AVIÃO,REGULAR,AFONSO PENA,ADALBERTO MENDES DA SILVA,ARREMETIDA NO AR,REGULAR,LEVE,0


#### Tratando missings identificados por "***"

In [8]:
# Loop sobre colunas:
for c in list(aero.columns):
    # Checando a existência de missings identificados por "***":
    if '***' in list(aero[c].unique()):
        # Modificando a definição do missing:
        aero[c] = aero[c].apply(lambda x: 'missing_value' if x=='***' else x)

#### Train-test split

In [9]:
aero_train = aero[aero.codigo_ocorrencia2.isin(list(ocorr_train.codigo_ocorrencia))]
aero_test = aero[aero.codigo_ocorrencia2.isin(list(ocorr_test.codigo_ocorrencia))]

<a id='feat_eng'></a>

### Feature engineering

#### Características das aeronaves

Criando variáveis binárias

In [10]:
# Relação de variáveis categóricas:
aero_cat_vars = ['aeronave_tipo_veiculo', 'aeronave_fabricante', 'aeronave_motor_tipo',
                 'aeronave_motor_quantidade', 'aeronave_pais_registro', 'aeronave_registro_segmento',
                 'aeronave_voo_origem', 'aeronave_voo_destino', 'aeronave_fase_operacao', 'aeronave_tipo_operacao']

# Aplicando one-hot encoding sobre variáveis categóricas:
transf_data = applying_one_hot(aero_train, aero_cat_vars, test_data=aero_test, variance_param=-1)

# Dados de treino:
aero_cat_vars_train = transf_data['training_data']
aero_cat_vars_train = aero_cat_vars_train[[c for c in aero_cat_vars_train.columns if ('C#' in c) |
                                           (c=='codigo_ocorrencia2')]]

# Dados de teste:
aero_cat_vars_test = transf_data['test_data']
aero_cat_vars_test = aero_cat_vars_test[[c for c in aero_cat_vars_test.columns if ('C#' in c) |
                                         (c=='codigo_ocorrencia2')]]

/home/matheus.rosso/Downloads/aero/Codes/transformations.py:397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data[f] = training_data[f].apply(text_clean)
/home/matheus.rosso/Downloads/aero/Codes/transformations.py:403: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kwargs[d][f] = kwargs[d][f].apply(text_clean)
/home/matheus.rosso/Downloads/aero/Codes/transformations.py:158: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

Number of categorical features: 10
Number of overall selected dummies: 1321.


Agregando variáveis por ocorrência

In [11]:
# Agregando as variáveis binárias por ocorrência:
aero_cat_vars_train = aero_cat_vars_train.groupby('codigo_ocorrencia2').mean().reset_index(drop=False)
aero_cat_vars_test = aero_cat_vars_test.groupby('codigo_ocorrencia2').mean().reset_index(drop=False)

# Loop sobre colunas:
for c in [c for c in aero_cat_vars_train.columns if c!='codigo_ocorrencia2']:
    # Tratando os valores:
    aero_cat_vars_train[c] = aero_cat_vars_train[c].apply(lambda x: 1 if x > 0 else 0)
    
# Loop sobre colunas:
for c in [c for c in aero_cat_vars_test.columns if c!='codigo_ocorrencia2']:
    # Tratando os valores:
    aero_cat_vars_test[c] = aero_cat_vars_test[c].apply(lambda x: 1 if x > 0 else 0)
    
# Seleção de variáveis de acordo com a variância:
vars_to_keep = [v for v in [c for c in aero_cat_vars_train.columns if c!='codigo_ocorrencia2'] if
                aero_cat_vars_train[v].var() > 0.01]
vars_to_keep.insert(0, 'codigo_ocorrencia2')
    
# Agregando variáveis contínuas por ocorrência:
cont_vars_train = aero_train.groupby('codigo_ocorrencia2').mean()[['aeronave_assentos']].reset_index(drop=False)
cont_vars_test = aero_test.groupby('codigo_ocorrencia2').mean()[['aeronave_assentos']].reset_index(drop=False)

# Merge entre os datasets de características de aeronave:
aero_train_transf = aero_cat_vars_train[vars_to_keep].merge(cont_vars_train, on='codigo_ocorrencia2', how='inner')
aero_test_transf = aero_cat_vars_test[vars_to_keep].merge(cont_vars_test, on='codigo_ocorrencia2', how='inner')

#### Idade da aeronave

In [12]:
# Merge com o dataset de ocorrências:
age_aero_train = aero_train[['codigo_ocorrencia2', 'aeronave_ano_fabricacao']].\
                    merge(ocorr_train[['codigo_ocorrencia2', 'ocorrencia_dia']],
                          on='codigo_ocorrencia2', how='left')
age_aero_test = aero_test[['codigo_ocorrencia2', 'aeronave_ano_fabricacao']].\
                    merge(ocorr_test[['codigo_ocorrencia2', 'ocorrencia_dia']],
                          on='codigo_ocorrencia2', how='left')

# Cálculo da idade da aeronave:
age_aero_train['idade'] = age_aero_train[['aeronave_ano_fabricacao',
                                          'ocorrencia_dia']].apply(lambda x:
                                                                   x['ocorrencia_dia'].year-x['aeronave_ano_fabricacao'],
                                                                   axis=1)
age_aero_test['idade'] = age_aero_test[['aeronave_ano_fabricacao',
                                        'ocorrencia_dia']].apply(lambda x:
                                                                 x['ocorrencia_dia'].year-x['aeronave_ano_fabricacao'],
                                                                 axis=1)

# Merge entre os datasets de características de aeronave:
aero_train_transf = aero_train_transf.merge(age_aero_train[['codigo_ocorrencia2', 'idade']],
                                            on='codigo_ocorrencia2', how='inner')
aero_test_transf = aero_test_transf.merge(age_aero_test[['codigo_ocorrencia2', 'idade']],
                                          on='codigo_ocorrencia2', how='inner')

#### Merge com o dataset de ocorrências:

In [13]:
ocorr_train = ocorr_train.merge(aero_train_transf, on='codigo_ocorrencia2', how='left')
ocorr_test = ocorr_test.merge(aero_test_transf, on='codigo_ocorrencia2', how='left')

<a id='data_modeling'></a>

## Modelagem

In [14]:
# Parâmetros para a seleção de features:
selection_params = {
    'method': 'supervised', 'threshold': 0,
    'estimator': LogisticRegression(C=1.0, penalty='l1', solver='liblinear')
}

# Dicionário para model assessment:
model_assess = {}

<a id='pre_proc'></a>

### Pré-processamento dos dados

In [15]:
# Pré-processamento dos dados:
ocorr_train, ocorr_test, ocorr_train_scaled, ocorr_test_scaled, stats = pre_process(training_data=ocorr_train,
                                                                                    test_data=ocorr_test,
                                                                                    vars_to_drop=drop_vars,
                                                                                    log_transform=numerical_transf,
                                                                                    standardize=numerical_transf)

---------------------------------------------------------------------------------------------------------
CLASSIFYING FEATURES AND EARLY SELECTION


Initial number of features: 79.
0 features were dropped for excessive number of missings!
0 features were dropped for having no variance!
79 remaining features.


---------------------------------------------------------------------------------------------------------


---------------------------------------------------------------------------------------------------------
ASSESSING MISSING VALUES


Training data:
Number of features with missings: 9 out of 97 features (9.28%).
Average number of missings: 77 out of 4326 observations (1.78%).

Test data:
Number of features with missings: 6 out of 97 features (6.19%).
Average number of missings: 24 out of 1862 observations (1.29%).


---------------------------------------------------------------------------------------------------------


----------------------------------------------------

In [16]:
# Dados de treinamento e de teste:
X_train, y_train = (ocorr_train_scaled.drop(drop_vars, axis=1), ocorr_train_scaled['y'])
X_test, y_test = (ocorr_test_scaled.drop(drop_vars, axis=1), ocorr_test_scaled['y'])

<a id='lr'></a>

### Regressão logística

In [17]:
# Identificação da estimação:
estimation_id = str(int(time.time()))

# Grids de hiper-parâmetros:
grid_param = {'C': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.25, 0.3, 0.5, 0.75, 1, 3, 10]}
default_param = {'C': 1.0}
fixed_params = {'penalty':'l1', 'solver':'liblinear', 'warm_start':True}

# Criando o objeto de K-folds CV fit:
model_lr = Kfolds_fit(task='classification', method='logistic_regression',
                      metric='roc_auc', num_folds=3, shuffle=False,
                      pre_selecting=False, pre_selecting_params=selection_params,
                      random_search=False, grid_param=grid_param, default_param=default_param,
                      fixed_params=fixed_params)

# Treinando o modelo:
model_lr.fit(train_inputs=X_train, train_output=y_train, test_inputs=X_test, test_output=y_test)

---------------------------------------------------------------------
Train-test estimation outcomes:


Outcomes from K-folds CV estimation:
   Number of data folds: 3.
   Estimation method: logistic regression.
   Metric for choosing best hyper-parameter: roc_auc.
   Best hyper-parameters: {'C': 1}.
   CV performance metric associated with best hyper-parameters: 0.8805.


Performance metrics evaluated at test data:
   test_roc_auc = 0.8444
   test_prec_avg = 0.6318
   test_brier = 0.1665
---------------------------------------------------------------------


------------------------------------
Running time: 0.07 minutes.
Start time: 2022-01-02, 15:10:34
End time: 2022-01-02, 15:10:38
------------------------------------


#### Model assessment

In [18]:
model_assess[estimation_id] = {
    'estimation_id': estimation_id,
    'n_obs_train': len(X_train),
    'n_obs_test': len(X_test),
    'train_interval': (str(ocorr_train['ocorrencia_dia'].min().date()),
                       str(ocorr_train['ocorrencia_dia'].max().date())),
    'test_interval': (str(ocorr_test['ocorrencia_dia'].min().date()),
                      str(ocorr_test['ocorrencia_dia'].max().date())),
    'avg_y_train': y_train.mean(),
    'avg_y_test': y_test.mean(),
    'n_vars_train': X_train.shape[1],
    'n_vars_test': X_test.shape[1],
    'numerical_transf': numerical_transf,
    'comment': comment,
    'method': 'LR',
    'performance_metrics': model_lr.performance_metrics
}

#### Feature importances

In [19]:
# Coeficiente estimado para cada feature:
feat_importances_lr = pd.DataFrame(data={
    'feature': list(X_train.columns),
    'beta': list(model_lr.model.coef_[0]),
    'abs_beta': [abs(b) for b in list(model_lr.model.coef_[0])]
})

feat_importances_lr.sort_values('abs_beta', ascending=False).head(10)

,feature,beta,abs_beta
89,C#ocorrencia_cidade#RIBEIRAO_PRETO,-2.795677,2.795677
61,C#aeronave_fase_operacao#TAXI,-2.614535,2.614535
55,C#aeronave_fase_operacao#ESTACIONAMENTO,-2.251580,2.251580
69,C#aeronave_tipo_operacao#REGULAR,-1.939718,1.939718
63,C#aeronave_tipo_operacao#AGRICOLA,1.814772,1.814772
21,C#aeronave_fabricante#SIKORSKY_AIRCRAFT,-1.721227,1.721227
97,C#ocorrencia_uf#ES,-1.695939,1.695939
76,C#ocorrencia_cidade#BELO_HORIZONTE,-1.665141,1.665141
92,C#ocorrencia_cidade#SAO_PAULO,-1.590181,1.590181
85,C#ocorrencia_cidade#LONDRINA,-1.512565,1.512565


<a id='svm'></a>

### SVM

In [20]:
# Identificação da estimação:
estimation_id = str(int(time.time()))

# Grids de hiper-parâmetros:
params = {'C': [1],
          'kernel': ['poly'],
          'degree': [1, 2, 3, 4],
          'gamma': ['scale']}
params_default = {'C': 1.0, 'kernel': 'poly', 'degree': 1, 'gamma': 'scale'}
fixed_params = {'probability': True}

# Criando o objeto de K-folds CV fit:
model_svm = Kfolds_fit(task='classification', method='SVM',
                       metric='roc_auc', num_folds=3, random_search=False, shuffle=False,
                       pre_selecting=False, pre_selecting_params=selection_params,
                       grid_param=params, default_param=params_default, fixed_params=fixed_params,
                       parallelize=False)

# Treinando o modelo:
model_svm.fit(train_inputs=X_train, train_output=y_train, test_inputs=X_test, test_output=y_test)

---------------------------------------------------------------------
Train-test estimation outcomes:


Outcomes from K-folds CV estimation:
   Number of data folds: 3.
   Estimation method: SVM.
   Metric for choosing best hyper-parameter: roc_auc.
   Best hyper-parameters: {'C': 1, 'kernel': 'poly', 'degree': 2, 'gamma': 'scale'}.
   CV performance metric associated with best hyper-parameters: 0.8717.


Performance metrics evaluated at test data:
   test_roc_auc = 0.8436
   test_prec_avg = 0.6431
   test_brier = 0.1652
---------------------------------------------------------------------


------------------------------------
Running time: 0.31 minutes.
Start time: 2022-01-02, 15:10:38
End time: 2022-01-02, 15:10:57
------------------------------------


#### Model assessment

In [21]:
model_assess[estimation_id] = {
    'estimation_id': estimation_id,
    'n_obs_train': len(X_train),
    'n_obs_test': len(X_test),
    'train_interval': (str(ocorr_train['ocorrencia_dia'].min().date()),
                       str(ocorr_train['ocorrencia_dia'].max().date())),
    'test_interval': (str(ocorr_test['ocorrencia_dia'].min().date()),
                      str(ocorr_test['ocorrencia_dia'].max().date())),
    'avg_y_train': y_train.mean(),
    'avg_y_test': y_test.mean(),
    'n_vars_train': X_train.shape[1],
    'n_vars_test': X_test.shape[1],
    'numerical_transf': numerical_transf,
    'comment': comment,
    'method': 'SVM',
    'performance_metrics': model_svm.performance_metrics
}

<a id='lightgbm'></a>

### LightGBM

In [22]:
# Identificação da estimação:
estimation_id = str(int(time.time()))

# Grids de hiper-parâmetros:
grid_param = {'bagging_fraction': uniform(0.5, 0.5),
              'learning_rate': uniform(0.0001, 0.1),
              'max_depth': randint(1, 10),
              'num_iterations': [100, 250, 500]}
default_param = {'bagging_fraction': 0.75, 'learning_rate': 0.01, 'max_depth': 10, 'num_iterations': 500}

# Criando o objeto de K-folds CV fit:
model_lgbm = Kfolds_fit(task='binary', method='light_gbm',
                        metric='roc_auc', num_folds=3, shuffle=False,
                        pre_selecting=False, pre_selecting_params=selection_params,
                        random_search=True, n_samples=10,
                        grid_param=grid_param, default_param=default_param)

# Treinando o modelo:
model_lgbm.fit(train_inputs=X_train, train_output=y_train, test_inputs=X_test, test_output=y_test)

/home/matheus.rosso/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/matheus.rosso/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/matheus.rosso/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/matheus.rosso/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/matheu

---------------------------------------------------------------------
Train-test estimation outcomes:


Outcomes from K-folds CV estimation:
   Number of data folds: 3.
   Number of samples for random search: 10.
   Estimation method: light gbm.
   Metric for choosing best hyper-parameter: roc_auc.
   Best hyper-parameters: {'bagging_fraction': 0.781003033650032, 'learning_rate': 0.09066127436435546, 'max_depth': 3, 'num_iterations': 250}.
   CV performance metric associated with best hyper-parameters: 0.8831.


Performance metrics evaluated at test data:
   test_roc_auc = 0.8518
   test_prec_avg = 0.6545
   test_brier = 0.1619
---------------------------------------------------------------------


------------------------------------
Running time: 0.1 minutes.
Start time: 2022-01-02, 15:10:57
End time: 2022-01-02, 15:11:03
------------------------------------


#### Model assessment

In [23]:
model_assess[estimation_id] = {
    'estimation_id': estimation_id,
    'n_obs_train': len(X_train),
    'n_obs_test': len(X_test),
    'train_interval': (str(ocorr_train['ocorrencia_dia'].min().date()),
                       str(ocorr_train['ocorrencia_dia'].max().date())),
    'test_interval': (str(ocorr_test['ocorrencia_dia'].min().date()),
                      str(ocorr_test['ocorrencia_dia'].max().date())),
    'avg_y_train': y_train.mean(),
    'avg_y_test': y_test.mean(),
    'n_vars_train': X_train.shape[1],
    'n_vars_test': X_test.shape[1],
    'numerical_transf': numerical_transf,
    'comment': comment,
    'method': 'LightGBM',
    'performance_metrics': model_lgbm.performance_metrics
}

#### Feature importances

In [24]:
# Feature importances:
feat_importances_gbm = pd.DataFrame(data={
    'feature': list(X_train.columns),
    'importance': list(model_lgbm.model.feature_importance())
})

feat_importances_gbm.sort_values('importance', ascending=False).head(10)

,feature,importance
72,L#idade,228
71,L#aeronave_assentos,115
46,C#aeronave_voo_destino#FORA_DE_AERODROMO,75
43,C#aeronave_voo_origem#FORA_DE_AERODROMO,71
111,C#ocorrencia_saida_pista#NAO,44
61,C#aeronave_fase_operacao#TAXI,29
59,C#aeronave_fase_operacao#POUSO,29
68,C#aeronave_tipo_operacao#PRIVADA,28
1,C#aeronave_tipo_veiculo#AVIAO,27
50,C#aeronave_fase_operacao#CORRIDA_APOS_POUSO,27


<a id='xgboost'></a>

### XGBoost

In [25]:
# Identificação da estimação:
estimation_id = str(int(time.time()))

# Grid de hiper-parâmetros
grid_param = {'subsample': uniform(0.5, 0.5),
              'eta': uniform(0.0001, 0.1),
              'max_depth': randint(1, 10),
              'num_boost_round': [100, 250, 500]}
default_param = {'subsample': 0.75, 'eta': 0.01, 'max_depth': 10, 'num_boost_round': 100}

# Criando o objeto de K-folds CV fit:
model_xgb = Kfolds_fit(task='binary:logistic', method='xgboost', num_folds=3, metric='roc_auc', shuffle=False,
                       random_search=True, n_samples=10,
                       grid_param=grid_param, default_param=default_param,
                       pre_selecting=False, pre_selecting_params=selection_params,
                       parallelize=False)

# Treinando o modelo:
model_xgb.fit(train_inputs=X_train, train_output=y_train, test_inputs=X_test, test_output=y_test)

[15:11:03] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:04] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:04] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[15:11:05] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:06] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:08] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[15:11:09] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:10] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:10] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[15:11:10] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:12] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:14] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[15:11:16] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:17] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:18] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[15:11:20] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:21] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:22] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[15:11:23] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:24] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:25] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[15:11:26] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:27] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:27] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[15:11:28] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:29] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:11:29] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[15:11:30] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
---------------------------------------------------------------------
Train-test estimation outcomes:


Outcomes from K-folds CV estimation:
   Number of data folds: 3.
   Number of samples for random search: 10.
   Estimation method: xgboost.
   Metric for choosing best hyper-parameter: roc_auc.
   Best hyper-parameters: {'subsample': 0.9444406041761167, 'eta': 0.025271558565476228, 'max_depth': 5, 'num_boost_round': 500}.
   CV performance metric associated with best hyper-parameters: 0.8807.


Performance metrics evaluated at test data:
   test_roc_auc = 0.8531
   test_prec_avg = 0.6565
   test_brier = 0.1611
---------------------------------------------------------------------


------------------------------------
Running time: 0.47 

#### Model assessment

In [26]:
model_assess[estimation_id] = {
    'estimation_id': estimation_id,
    'n_obs_train': len(X_train),
    'n_obs_test': len(X_test),
    'train_interval': (str(ocorr_train['ocorrencia_dia'].min().date()),
                       str(ocorr_train['ocorrencia_dia'].max().date())),
    'test_interval': (str(ocorr_test['ocorrencia_dia'].min().date()),
                      str(ocorr_test['ocorrencia_dia'].max().date())),
    'avg_y_train': y_train.mean(),
    'avg_y_test': y_test.mean(),
    'n_vars_train': X_train.shape[1],
    'n_vars_test': X_test.shape[1],
    'numerical_transf': numerical_transf,
    'comment': comment,
    'method': 'XGBoost',
    'performance_metrics': model_xgb.performance_metrics
}

<a id='model_selection'></a>

### Model selection

In [31]:
pd.DataFrame(data={
    'estimation_id': [model_assess[es_id]['estimation_id'] for es_id in model_assess],
    'n_obs_train': [model_assess[es_id]['n_obs_train'] for es_id in model_assess],
    'n_obs_test': [model_assess[es_id]['n_obs_test'] for es_id in model_assess],
    'train_interval': [model_assess[es_id]['train_interval'] for es_id in model_assess],
    'test_interval': [model_assess[es_id]['test_interval'] for es_id in model_assess],
    'avg_y_train': [model_assess[es_id]['avg_y_train'] for es_id in model_assess],
    'avg_y_test': [model_assess[es_id]['avg_y_test'] for es_id in model_assess],
    'n_vars_train': [model_assess[es_id]['n_vars_train'] for es_id in model_assess],
    'n_vars_test': [model_assess[es_id]['n_vars_test'] for es_id in model_assess],
    'method': [model_assess[es_id]['method'] for es_id in model_assess],
    'test_roc_auc': [model_assess[es_id]['performance_metrics']['test_roc_auc'] for es_id in model_assess],
    'test_prec_avg': [model_assess[es_id]['performance_metrics']['test_prec_avg'] for es_id in model_assess],
    'test_brier': [model_assess[es_id]['performance_metrics']['test_brier'] for es_id in model_assess]
}).sort_values('test_roc_auc', ascending=False)

,estimation_id,n_obs_train,n_obs_test,train_interval,test_interval,avg_y_train,avg_y_test,n_vars_train,n_vars_test,method,test_roc_auc,test_prec_avg,test_brier
3,1641147063,4326,1862,"(2010-01-03, 2017-12-03)","(2017-12-06, 2021-08-18)",0.319464,0.301289,138,138,XGBoost,0.853072,0.656459,0.161125
2,1641147057,4326,1862,"(2010-01-03, 2017-12-03)","(2017-12-06, 2021-08-18)",0.319464,0.301289,138,138,LightGBM,0.851791,0.654457,0.161925
0,1641147034,4326,1862,"(2010-01-03, 2017-12-03)","(2017-12-06, 2021-08-18)",0.319464,0.301289,138,138,LR,0.844413,0.631759,0.166467
1,1641147038,4326,1862,"(2010-01-03, 2017-12-03)","(2017-12-06, 2021-08-18)",0.319464,0.301289,138,138,SVM,0.843643,0.643145,0.165225


#### Model evaluation

Matthews correlation coefficient (MCC) para definição de limiar

In [44]:
# MCC por limiar:
pd.DataFrame(data={
    'threshold': np.linspace(0,1, 100),
    'mcc': [matthews_corrcoef(model_xgb.test_scores['y_true'],
                              model_xgb.test_scores['test_score'].apply(lambda x: 1 if x > t else 0))
            for t in np.linspace(0,1, 100)]}).sort_values('mcc', ascending=False).head(10)

/home/matheus.rosso/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/matheus.rosso/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/matheus.rosso/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/matheus.rosso/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


,threshold,mcc
55,0.555556,0.544934
54,0.545455,0.539649
57,0.575758,0.537538
56,0.565657,0.536892
51,0.515152,0.531161
53,0.535354,0.529800
49,0.494949,0.529631
48,0.484848,0.528202
50,0.505051,0.527990
52,0.525253,0.526501


Matriz de confusão

In [56]:
test_acc = accuracy_score(model_xgb.test_scores['y_true'],
                          model_xgb.test_scores['test_score'].apply(lambda x: 1 if x > 0.555556 else 0))

print(f'Acurácia: {test_acc*100:.0f}%.')
confusion_matrix(model_xgb.test_scores['y_true'],
                 model_xgb.test_scores['test_score'].apply(lambda x: 1 if x > 0.555556 else 0))

Acurácia: 78%.


array([[1004,  297],
       [ 106,  455]])